# Building Trees using scikit-learn - Lab

## Introduction

Following the toy example we saw in the previous lesson, we'll now build a decision tree for a more complex dataset. This lab covers all major areas of standard machine learning practice , from data acquisition to evaluation of results. We'll continue to use the scikit-learn and pandas libraries to conduct this analysis, following the same structure we saw in the previous lesson.

## Objectives

You will be able to:

- Use pandas to prepare the data for the scikit-learn decision tree algorithm
- Train the classifier with a training dataset and evaluate performance using different measures
- Visualize the decision tree and interpret the visualization

## UCI Banknote Authentication Data Set

In this lab, we'll work with a popular dataset for classification called the "UCI Bank Note Authentication Dataset'. This Data were extracted from images that were taken from genuine and forged banknotes! The notes were first digitized, followed by a numerical transformation using DSP techniques. The final set of engineered features are all continuous in nature, meaning that our dataset consists entirely of floats, with no strings to worry about. If you're curious about how the dataset was created, you can visit the UCI link listed above to learn about feature engineering in detail!

We have following attributes in the dataset. 

1. __Variance__ of Wavelet Transformed image (continuous) 
2. __Skewness__ of Wavelet Transformed image (continuous) 
3. __Curtosis__ of Wavelet Transformed image (continuous) 
4. __Entropy__ of image (continuous) 
5. __Class__ (integer) - Target/Label 

## Step 1: Import necessary Libraries
- Import necessary libraries as we saw in previous lesson

In [1]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import tree 
from sklearn.preprocessing import OneHotEncoder
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pandas as pd 
import numpy as np 

## Step 2: Import Data

Now, we'll load our dataset in a DataFrame, perform some basic EDA, and generally get a feel for the data we'll be working with.

- Read the file `"data_banknote_authentication.csv"` as a pandas dataframe. Note that there is no header information in this dataset.
- Assign column names 'Variance', 'Skewness', 'Curtosis', 'Entropy', 'Class' to dataset in the given order.
- View the basic statistics and shape of dataset.
- Check for frequency of positive and negative examples in the target variable

In [2]:
# Create Dataframe
df = pd.read_csv('data_banknote_authentication.csv')

In [3]:
# Describe the dataset
df.head()

,3.6216,8.6661,-2.8073,-0.44699,0
0,4.54590,8.1674,-2.4586,-1.46210,0
1,3.86600,-2.6383,1.9242,0.10645,0
2,3.45660,9.5228,-4.0112,-3.59440,0
3,0.32924,-4.4552,4.5718,-0.98880,0
4,4.36840,9.6718,-3.9606,-3.16250,0


In [4]:
# Shape of dataset
df.columns = ['Variance', 'Skewness', 'Curtosis', 'Entropy', 'Class']
df.info()
#5 columns, 1371 rows

df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 5 columns):
Variance    1371 non-null float64
Skewness    1371 non-null float64
Curtosis    1371 non-null float64
Entropy     1371 non-null float64
Class       1371 non-null int64
dtypes: float64(4), int64(1)
memory usage: 53.6 KB


,Variance,Skewness,Curtosis,Entropy,Class
count,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000
mean,0.431410,1.917434,1.400694,-1.192200,0.444931
std,2.842494,5.868359,4.310105,2.101683,0.497139
min,-7.042100,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.774700,-1.711300,-1.553350,-2.417000,0.000000
50%,0.495710,2.313400,0.616630,-0.586650,0.000000
75%,2.814650,6.813100,3.181600,0.394810,1.000000
max,6.824800,12.951600,17.927400,2.449500,1.000000


In [5]:
# Class frequency of target variable 
df.Class.value_counts()
# 0:761
# 1:610 - relatively even representation!

0    761
1    610
Name: Class, dtype: int64

## Step 3: Create Features and Labels, Training and Test Data

Now we need to create our feature set `X` and labels `y`. 
- Create `X` and `y` by selecting the appropriate columns from the dataset
- Create a 80/20 split on the dataset for training/testing. Use `random_state=10` for reproducibility

In [6]:
# Create features and labels
X = df.loc[:,['Variance', 'Skewness', 'Curtosis', 'Entropy']]
y = df.loc[:, 'Class']


In [7]:
# Perform an 80/20 split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 10)

## Step 4: Train the Classifier and Make Predictions
- Create an instance of decision tree classifier with random_state=10 for reproducibility
- Fit the training data to the model 
- USe the trained model to make predictions with test data

In [8]:
# Train a DT classifier
tree = DecisionTreeClassifier(random_state =10)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=10,
            splitter='best')

In [9]:
# Make predictions for test data

y_preds = tree.predict(X_test)

## Step 5: Check Predictive Performance

We can now use different evaluation measures to check the predictive performance of the classifier. 
- Check the accuracy , AUC and create a confusion matrix 
- Interpret the results 

In [10]:
# Calculate Accuracy , AUC and Confusion matrix 

def metrics(labels,predictions):
        print('Accuracy score:{}'.format(accuracy_score(labels,predictions)))

   # Check the AUC for predictions
        false_positive_rate, true_positive_rate, thresholds = roc_curve(labels, predictions)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        print("\nAUC is :{0}".format(round(roc_auc,2)))

# Create and print a confusion matrix 
        print('\nConfusion Matrix')
        print('----------------')
        matrix =pd.crosstab(labels, predictions, rownames=['True'], colnames=['Predicted'], margins=True)   
        print(matrix)
        
        
        
        
metrics(y_test,y_preds)

Accuracy score:0.9854545454545455

AUC is :0.99

Confusion Matrix
----------------
Predicted    0    1  All
True                    
0          152    4  156
1            0  119  119
All        152  123  275


## Bonus: Re-grow the Tree Using Entropy 

SO in the above example, we used all default settings for decision tree classifier. The default impurity criterion in scikit-learn is the Gini impurity. We can change it back to entropy by passing in `criterion='entropy'` argument to the classifier in the training phase. 
- Repeat the above tasks for training, evaluation and visualization using Entropy measure. (
- Compare and interpret the results 

In [11]:
# Train a DT classifier
Tree = DecisionTreeClassifier(random_state =10, criterion ='entropy')
Tree.fit(X_train, y_train)



y_preds = Tree.predict(X_test)

        
metrics(y_test,y_preds)

Accuracy score:0.9854545454545455

AUC is :0.99

Confusion Matrix
----------------
Predicted    0    1  All
True                    
0          152    4  156
1            0  119  119
All        152  123  275


In [12]:
pip install pydotplus

Note: you may need to restart the kernel to use updated packages.


In [15]:
from IPython.display import Image  
from sklearn import tree
import pydotplus

# Create DOT data
#dot_data = tree.export_graphviz(Tree, out_file=None, 
                               # feature_names=['Variance', 'Skewness', 'Curtosis', 'Entropy'],  
                               # class_names='class')

# Draw graph
#graph = pydotplus.graph_from_dot_data(dot_data)  

dot_data = tree.export_graphviz(Tree)
graph = pydotplus.graphviz.graph_from_dot_data(dot_data)



Image(graph.create_png())

InvocationException: GraphViz's executables not found

In [14]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


## Level up - Optional 

- We discussed earlier that decision trees are very sensitive towards outliers. Try to identify and remove/fix any possible outliers in the dataset. 
- Check the distributions of the data. Is there any room for normalization/scaling of data ? Apply these techniques and see if it improves upon accuracy score. 

## Summary 

In this lesson, we looked at growing a decision tree for banknote authentication dataset which is composed of extracted continuous features from photographic data. We looked at different stages of the experiment including data acquisition, training, prediction and evaluation. We also looked at growing trees using entropy vs. gini impurity criteria. In following lessons, we shall look at some more such pre-train tuning techniques for ensuring an optimal classifier for learning and prediction.  